In [1]:
import sys
import re
import requests
from lxml import html

sys.path.append("../../")

import penaltyblog as pb

In [2]:
sofifa = pb.scrapers.SoFifa()

In [3]:
player_info = sofifa.get_player_info(max_pages=1)

In [4]:
player_info.head()

,name,url,nationality,team,age,overall_rating,potential,value,wage,total,positions
id,,,,,,,,,,,
246764,Mohamed Camara,/player/246764/mohamed-camara/220067/,Mali,FC Red Bull Salzburg,21,76,84,€16M,€18K,2003,"[CDM, CM]"
208574,Filip Kostić,/player/208574/filip-kostic/220067/,Serbia,Eintracht Frankfurt,28,84,84,€42.5M,€52K,2229,"[LM, LWB]"
241811,Sergio Gómez Martín,/player/241811/sergio-gomez-martin/220067/,Spain,RSC Anderlecht,20,72,81,€4.9M,€11K,2014,"[LB, RM, LM]"
257319,Alan Virginius,/player/257319/alan-virginius/220067/,France,FC Sochaux-Montbéliard,18,65,82,€1.8M,€700,1530,[RM]
253124,Matheus Luiz Nunes,/player/253124/matheus-luiz-nunes/220067/,Portugal,Sporting CP,22,79,86,€35.5M,€15K,2102,[CM]


In [5]:
url = "https://sofifa.com/player/241811/sergio-gomez-martin/220067/"

r = requests.get(url)

content = r.content
tree = html.fromstring(content)

In [6]:
parser = html.HTMLParser(encoding=r.encoding)
tree = html.document_fromstring(content, parser=parser)

In [7]:
tmp = dict()
tmp["name"] = tree.xpath("//div[contains(@class, 'info')]/h1")[0].text

info = tree.xpath("//div[contains(@class, 'info')]/div")[0].text_content()
tmp["weight"] = int(re.search("[0-9]*kg", info).group(0).split("kg")[0])
tmp["height"] = int(re.search("[0-9]*cm", info).group(0).split("cm")[0])
tmp["age"] = int(re.search("[0-9]*y\.o", info).group(0).split("y.o")[0])
tmp["pos"] = [x.text for x in tree.xpath("//div[contains(@class, 'info')]/div/span")]

stats = tree.xpath("//div[contains(@class, 'player')]//div[contains(@class, 'block-quarter')]")
for stat in stats[:2]:
    v = int(stat.xpath("div/span")[0].text)
    k = stat.xpath("div/div")[0].text
    tmp[k] = v
    
for stat in stats[2:]:
    v = stat.xpath("div")[0].text
    k = stat.xpath("div/div/text()")[0]
    tmp[k] = v    

tmp

{'name': 'Sergio Gómez Martín',
 'weight': 68,
 'height': 171,
 'age': 20,
 'pos': ['LB', 'RM', 'LM'],
 'Overall Rating': 72,
 'Potential': 81,
 'Value': '€4.9M',
 'Wage': '€11K'}

In [64]:
lis = tree.xpath("//div[contains(@class, 'block-quarter')]//ul[contains(@class, 'pl')]/li")
for li in lis:
    label = li.find("label")
    if label is None:
        continue
    label = li.xpath("label/text()")[0]
    
    span = li.find("span")
    if span is not None:
        span = li.xpath("span/text()")[0]
    else:
        span = li.xpath("text()")[0].strip()
    print(label, span)


Preferred Foot Left
Weak Foot 3
Skill Moves 4
International Reputation 1
Work Rate High/ Medium
Body Type Normal (170-185)
Real Face Yes
Release Clause €8.3M
ID 241811
Position LB
Kit Number 17
Joined Jul 1, 2021
Contract Valid Until 2025


In [59]:
li.xpath("text()")

['Left']